# 🗣️ Ferramenta de transcrição de áudio e análise de conteúdo em pesquisas qualitativas

### Gabriel de Antonio Mazetto  
**Projeto de Iniciação Científica - PIBIC/CNPq/INPE**  
- Orientadora: Dra. Minella Alves Martins (INPE)
- Co-orientadora: Dra. Maria Paula Pires de Oliveira (PUC-Campinas)  
- Colaboradora: Dra Denise Helena Lombardo Ferreira (PUC-Campinas)
- Período: Agosto/2024 — Agosto/2025  

**Apoio Institucional:**   
- Ferramenta elaborado com apoio do Conselho Nacional de Desenvolvimento Científico e Tecnológico (bolsa PIBIC/CNPq/INPE) e da Coordenação de Aperfeiçoamento de Pessoal de Nível Superior - Brasil (CAPES) – Código de Financiamento 001.

---

## 🎯 Objetivo Geral

Este notebook foi desenvolvido como ferramenta de apoio à análise de conteúdo em pesquisas qualitativas. Ele automatiza etapas como transcrição, exploração semântica e comparação de múltiplas entrevistas, depoimentos e outros registros textuais.

---

## 🛠️ Funcionalidades

- **Transcrição automática** de arquivos de áudio e vídeo utilizando o modelo **Whisper (OpenAI)**;
- **Nuvens de palavras e gráficos de frequência**, para explorar os termos mais recorrentes;
- **Grafo de coocorrência de palavras**, visualizando relações entre termos em uma rede semântica;
- **Reconhecimento de Entidades Nomeadas (NER)**: identificação de pessoas, locais e organizações mencionados nos discursos;
- **Codificação.**
- **Buscas interativas** no conteúdo transcrito, com diferentes abordagens:
  - **Busca exata (booleana)** por termos específicos;
  - **Busca por lema**, que considera diferentes formas de uma mesma palavra (ex: "comer", "comeu", "comendo");
  - **Busca por similaridade semântica de frases** e **similaridade de palavras**, utilizando embeddings gerados com o modelo `distiluse-base-multilingual-cased-v1` (via *SentenceTransformers*) — cada palavra é transformada em vetor, permitindo buscas por contexto e significado;
  - **Busca por palavras coocorrentes a um termo-alvo**, que complementa a análise do grafo de coocorrência ao destacar as palavras que frequentemente aparecem no mesmo contexto da palavra buscada.
---


In [ ]:
# @title ⚙️ 1. Instalando dependências e reiniciando o ambiente
# @markdown > Execute esta célula e aguarde até que a mensagem "✅ Instalações concluídas." e "‼️ REINICIANDO O AMBIENTE..." apareça.
# @markdown >
# @markdown > Após o reinício automático, você poderá seguir para a próxima etapa.

!pip install numpy==2.0.0 scipy==1.14.0 -q

!pip install git+https://github.com/openai/whisper.git -q
!pip install ffmpeg-python -q
!pip install pydrive2 -q

!pip install spacy -q
!pip install wordcloud -q

!pip install gensim -q
!pip install sentence-transformers -q
!pip install ipywidgets -q

!python -m spacy download pt_core_news_lg -q

from IPython.display import clear_output
clear_output()

print("✅ Instalações concluídas.")
print("‼️ REINICIANDO O AMBIENTE AUTOMATICAMENTE para carregar as novas bibliotecas...")

import time
time.sleep(1)

import os

os.kill(os.getpid(), 9)

In [ ]:
# @title 📂 2. Especifique a Origem e o(s) Arquivo(s)
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os
import re

# --- INSTRUÇÕES PARA O USUÁRIO ---
# @markdown ### 🔹 1. Selecione a Origem dos Arquivos
# @markdown Marque a caixa para buscar os arquivos no seu Google Drive. Se deixá-la desmarcada, o sistema procurará por arquivos que você subiu diretamente para o ambiente do Colab.
usar_google_drive = False #@param {type:"boolean"}

# @markdown ---
# @markdown ### 🔹 2. Defina os Arquivos
# @markdown **Se estiver usando o Google Drive:** Preencha o `nome_da_pasta`.
# @markdown - Para um ou mais **arquivos específicos**, preencha o `nome_do_arquivo` (para múltiplos arquivos, separe os nomes por vírgula).
# @markdown - Para **TODOS os arquivos** da pasta, marque a caixa `processar_todos_os_arquivos`.
# @markdown ---
# @markdown **Se estiver usando Arquivos Locais:** Deixe o `nome_da_pasta` em branco.
# @markdown - Para um ou mais **arquivos específicos**, preencha o `nome_do_arquivo` (para múltiplos arquivos, separe os nomes por vírgula).
# @markdown - Para **TODOS os arquivos** de mídia no ambiente, marque a caixa `processar_todos_os_arquivos`.

nome_da_pasta = ""  # @param {type:"string"}
nome_do_arquivo = ""  # @param {type:"string"}
processar_todos_os_arquivos = True #@param {type:"boolean"}
# -----------------------------------

file_paths = []
folder_path_stripped = nome_da_pasta.strip()
file_names_stripped = nome_do_arquivo.strip()

if usar_google_drive:
    # --- MÉTODO 1: BUSCAR NO GOOGLE DRIVE ---
    print("▶️ Modo Google Drive selecionado.")
    if not folder_path_stripped:
        print("❌ ERRO: O modo Google Drive está selecionado, mas o nome da pasta não foi preenchido.")
    else:
        try:
            auth.authenticate_user()
            gauth = GoogleAuth()
            gauth.credentials = GoogleCredentials.get_application_default()
            drive = GoogleDrive(gauth)

            # Navega pela estrutura de pastas
            path_parts = [part.strip() for part in folder_path_stripped.split('/') if part.strip()]
            parent_id = 'root'
            target_folder_id = None
            for i, part in enumerate(path_parts):
                query = f"'{parent_id}' in parents and title='{part}' and mimeType='application/vnd.google-apps.folder' and trashed=false"
                folder_list = drive.ListFile({'q': query}).GetList()
                if not folder_list:
                    print(f"❌ ERRO: A subpasta '{part}' não foi encontrada.")
                    target_folder_id = None
                    break
                parent_id = folder_list[0]['id']
                target_folder_id = parent_id

            if target_folder_id:
                print(f"✅ Pasta final '{folder_path_stripped}' encontrada.")
                file_list = []

                if processar_todos_os_arquivos:
                    print("Buscando todos os arquivos de mídia na pasta...")
                    query = f"'{target_folder_id}' in parents and (mimeType contains 'video/' or mimeType contains 'audio/') and trashed=false"
                    file_list = drive.ListFile({'q': query}).GetList()
                    if not file_list: print("❌ Nenhum arquivo de áudio ou vídeo encontrado na pasta.")

                else:
                    specific_filenames = [name.strip() for name in file_names_stripped.split(',') if name.strip()]
                    if not specific_filenames:
                         print("❌ ERRO: Nenhum nome de arquivo foi preenchido.")
                    else:
                        title_queries = []
                        for name in specific_filenames:
                            escaped_name = name.replace("'", "\\'")
                            title_queries.append(f"title='{escaped_name}'")
                        title_query_part = " or ".join(title_queries)
                        full_query = f"'{target_folder_id}' in parents and ({title_query_part}) and trashed=false"
                        print(f"Buscando pelos arquivos específicos: {specific_filenames}...")
                        file_list = drive.ListFile({'q': full_query}).GetList()

                # Faz o download dos arquivos encontrados
                if not file_list:
                     if not processar_todos_os_arquivos: print(f"❌ Nenhum dos arquivos especificados foi encontrado na pasta.")
                else:
                    print(f"Encontrado(s) {len(file_list)} arquivo(s). Iniciando download...")
                    for f in file_list:
                        print(f" -> Baixando '{f['title']}'...")
                        colab_file_path = os.path.join("/content/", f['title'])
                        f.GetContentFile(colab_file_path)
                        file_paths.append(colab_file_path)
                    print("✅ Todos os downloads foram concluídos!")

        except Exception as e:
            print(f"❌ Ocorreu um erro durante a conexão com o Google Drive: {e}")

else:
    # --- MÉTODO 2: USAR ARQUIVOS LOCAIS DO COLAB ---
    print("▶️ Modo Local selecionado.")
    if processar_todos_os_arquivos:
        print("Buscando todos os arquivos de mídia no ambiente local...")
        valid_extensions = ['.mp4', '.m4a', '.mp3', '.mov', '.wav', '.flac']
        all_local_files = [f for f in os.listdir('/content/') if os.path.isfile(os.path.join('/content/', f))]
        media_files = [f for f in all_local_files if any(f.lower().endswith(ext) for ext in valid_extensions)]

        if not media_files:
            print("❌ Nenhum arquivo de mídia compatível encontrado no ambiente do Colab.")
        else:
            for fname in media_files:
                file_paths.append(os.path.join("/content/", fname))
            print(f"✅ Arquivos locais de mídia encontrados: {', '.join(media_files)}")
    else:
        # Lógica para nomes específicos
        local_filenames = [name.strip() for name in file_names_stripped.split(',') if name.strip()]
        if not local_filenames:
            print("❌ ERRO: Nenhum nome de arquivo foi preenchido.")
        else:
            print(f"Verificando a existência de {len(local_filenames)} arquivo(s) locais...")
            for fname in local_filenames:
                local_path = os.path.join("/content/", fname)
                if os.path.exists(local_path):
                    file_paths.append(local_path)
                    print(f"  -> ✅ Arquivo '{fname}' encontrado.")
                else:
                    print(f"  -> ❌ ERRO: O arquivo '{fname}' não foi encontrado no ambiente do Colab.")

# --- Verificação final ---
if file_paths:
    print(f"\n➡️ {len(file_paths)} arquivo(s) pronto(s) para serem processados na próxima célula.")
else:
    print("\n⚠️ Nenhum arquivo foi definido para processamento. Verifique os erros acima.")


In [ ]:
# @title 🗣️ 3A. Transcrever Arquivo(s) de Áudio/Vídeo
# @markdown > Esta célula irá transcrever todos os arquivos definidos na Célula 2.

# @markdown > As transcrições a seguir foram geradas com o auxílio da ferramenta de inteligência artificial Whisper. Embora a tecnologia seja avançada, erros de transcrição podem ocorrer. Recomenda-se a verificação com o áudio original para garantir a máxima precisão das informações.

# @markdown ---
# @markdown **Modo de Análise para Múltiplos Arquivos:**
# @markdown Marque para juntar as transcrições em uma análise conjunta. Desmarque para analisar cada uma individualmente.
juntar_arquivos_para_analise_conjunta = False # @param {type:"boolean"}

import whisper
import pandas as pd
from google.colab import files
import os
import re
import time

def format_timestamp(seconds):
    try:
        milliseconds = round((seconds - int(seconds)) * 1000)
        td = pd.to_timedelta(seconds, unit='s')
        h, m, s = td.components.hours + (td.components.days * 24), td.components.minutes, td.components.seconds
        return f"{h:02d}:{m:02d}:{s:02d}.{milliseconds:03d}"
    except: return "00:00:00.000"

if 'file_paths' not in globals() or not file_paths:
    print("❌ ERRO: Nenhum arquivo para transcrever. Por favor, execute a Célula 2 primeiro.")
else:
    print("Carregando o modelo de transcrição (whisper medium)...")
    model = whisper.load_model("medium")

    if 'analises_individuais' in globals(): del analises_individuais
    if 'df_transcription' in globals(): del df_transcription

    all_transcription_data = []
    analises_individuais = {}
    time_offset = 0.0

    is_joint_analysis = juntar_arquivos_para_analise_conjunta or len(file_paths) == 1

    print(f"\nIniciando a transcrição de {len(file_paths)} arquivo(s)...")
    for path in file_paths:
        filename = os.path.basename(path)
        print("\n" + "="*50 + f"\nTranscrevendo: {filename}\n" + "="*50)

        result = model.transcribe(path, verbose=True, language='pt', fp16=False)

        current_file_data = []
        text_with_timestamps = ""
        for segment in result['segments']:
            start_f = format_timestamp(segment['start'])
            end_f = format_timestamp(segment['end'])
            text = segment['text'].strip()
            text_with_timestamps += f"[{start_f} --> {end_f}] {text}\n"
            current_file_data.append({"start": segment['start'], "end": segment['end'], "text": text})

        base_name, _ = os.path.splitext(filename)
        output_filename_ts = f"transcricao_com-tempo_{base_name}.txt"
        with open(output_filename_ts, 'w', encoding='utf-8') as f:
            f.write(text_with_timestamps)
        print(f"\n✅ Transcrição individual salva em '{output_filename_ts}'")

        output_filename_text = f"transcricao_texto-puro_{base_name}.txt"
        with open(output_filename_text, 'w', encoding='utf-8') as f:
            f.write(" ".join([item['text'] for item in current_file_data]))
        print(f"✅ Arquivo de texto puro '{output_filename_text}' salvo no ambiente.")

        print("Iniciando download do arquivo com marcação de tempo...")
        files.download(output_filename_ts)
        time.sleep(1)

        if is_joint_analysis:
            for item in current_file_data:
                item['start'] += time_offset
                item['end'] += time_offset
            all_transcription_data.extend(current_file_data)
            if all_transcription_data:
                time_offset = all_transcription_data[-1]['end']
        else:
            df_individual = pd.DataFrame(current_file_data)
            if not df_individual.empty:
                analises_individuais[filename] = {'df': df_individual, 'full_text': " ".join(df_individual['text'])}

    if is_joint_analysis:
        df_transcription = pd.DataFrame(all_transcription_data)
        full_text = " ".join(df_transcription['text'])
        juntar_arquivos_para_analise_conjunta = True
        if not df_transcription.empty:
            print("\n✅ Processamento concluído! Modo de Análise Conjunta ativado para as próximas células.")
    else:
        juntar_arquivos_para_analise_conjunta = False
        if analises_individuais:
             print(f"\n✅ Processamento concluído! {len(analises_individuais)} transcrições foram carregadas para análise individual.")

In [ ]:
# @title 📝 3B. Carregar Transcrição(ões) de Arquivo(s) .txt
# @markdown > Se você já transcreveu seus áudios em uma sessão anterior e baixou os arquivos `.txt`, use esta célula para carregá-los diretamente e pular a etapa de transcrição ao vivo.

# @markdown > **Instrução:** Suba os arquivos `.txt` para o ambiente do Colab (no painel 'Arquivos' à esquerda) e depois preencha as opções abaixo.

# @markdown > **Formatos Aceitos:**
# @markdown > * **Com Timestamp:** Arquivos padrão de transcrição, com o tempo de cada fala. Ex: `[00:00:01.234 --> 00:00:05.678] Texto da transcrição.`
# @markdown > * **Texto Simples:** Arquivos `.txt` contendo apenas o texto corrido. O conteúdo será automaticamente dividido em frases com base na pontuação (ponto, vírgula, etc.) e quebras de linha.
# @markdown ---
# @markdown ---
# @markdown **Opção 1: Listar Nomes dos Arquivos**

# @markdown Coloque os nomes dos arquivos .txt (separados por vírgula).
# @markdown *Exemplo: `transcricao_parte1.txt, transcricao_parte2.txt`*
transcript_filenames_str = "" # @param {type:"string"}

# @markdown ---
# @markdown **Opção 2: Processar Todos os Arquivos .txt**

# @markdown Se marcado, o campo acima será ignorado e o sistema buscará todos os arquivos `.txt` no ambiente.
processar_todos_os_arquivos_txt = True # @param {type:"boolean"}

# @markdown ---
# @markdown **Modo de Análise para Múltiplos Arquivos:**
# @markdown Marque para juntar os arquivos em uma análise conjunta. Desmarque para analisar cada um individualmente.
juntar_arquivos_para_analise_conjunta = False # @param {type:"boolean"}

import pandas as pd
import re
import os

# --- Função para converter timestamp (HH:MM:SS.ms) para segundos ---
def timestamp_to_seconds(ts_str):
    try:
        h, m, s_ms = ts_str.split(':')
        s, ms = s_ms.split('.')
        return int(h) * 3600 + int(m) * 60 + int(s) + int(ms) / 1000.0
    except ValueError: return 0.0
# ------------------------------------------------------------------

file_list = []
if processar_todos_os_arquivos_txt:
    print("▶️ Modo 'Processar Todos' selecionado. Buscando por arquivos .txt...")
    file_list = sorted([f for f in os.listdir('.') if f.endswith('.txt')])
    if not file_list:
        print("❌ Nenhum arquivo .txt foi encontrado no ambiente do Colab.")
    else:
        print(f"✅ Arquivos .txt encontrados: {', '.join(file_list)}")
else:
    print("▶️ Modo 'Nomes Específicos' selecionado.")
    file_list = [name.strip() for name in transcript_filenames_str.split(',') if name.strip()]
    if not file_list:
        print("❌ ERRO: Nenhum nome de arquivo foi inserido no campo de texto.")
# ---------------------------------------------------------


if file_list:
    if 'analises_individuais' in globals(): del analises_individuais
    if 'df_transcription' in globals(): del df_transcription

    if juntar_arquivos_para_analise_conjunta and len(file_list) > 1:
        print("\n▶️ Modo de Análise Conjunta selecionado.")
        all_transcription_data = []
        time_offset = 0.0
        processed_files, failed_files = [], []
        for filename in file_list:
            if not os.path.exists(filename):
                print(f"⚠️ ATENÇÃO: Arquivo '{filename}' não encontrado! Pulando...")
                failed_files.append(filename)
                continue
            print(f"Processando e juntando '{filename}'...")
            processed_files.append(filename)
            with open(filename, 'r', encoding='utf-8') as f:
                content = f.read()
                f.seek(0)

                has_timestamps = re.search(r'\[(\d{2}:\d{2}:\d{2}\.\d{3})\s-->\s(\d{2}:\d{2}:\d{2}\.\d{3})\]', content)

                if has_timestamps:
                    for line in f:
                        match = re.match(r'\[(\d{2}:\d{2}:\d{2}\.\d{3})\s-->\s(\d{2}:\d{2}:\d{2}\.\d{3})\]\s*(.*)', line)
                        if match:
                            start_str, end_str, text = match.groups()
                            all_transcription_data.append({
                                "start": timestamp_to_seconds(start_str) + time_offset,
                                "end": timestamp_to_seconds(end_str) + time_offset,
                                "text": text.strip()
                            })
                else:
                    print(f"   ℹ️ INFO: Arquivo '{filename}' não contém timestamps. O texto será dividido por frases.")
                    sentences = [s.strip() for s in re.split(r'[,.!?\n]+', content) if s.strip()]
                    for text in sentences:
                        all_transcription_data.append({
                            "start": 0.0 + time_offset,
                            "end": 0.0 + time_offset,
                            "text": text
                        })

            if all_transcription_data and has_timestamps:
                if all_transcription_data[-1]['end'] > time_offset:
                     time_offset = all_transcription_data[-1]['end']

        df_transcription = pd.DataFrame(all_transcription_data)
        if not df_transcription.empty:
            print("\n✅ Processamento concluído! Todos os arquivos foram unidos.")
        else:
            print("\n⚠️ Nenhum dado de transcrição foi carregado.")

    else:
        if len(file_list) > 1:
            print("\n▶️ Modo de Análise Individual selecionado.")
        analises_individuais = {}
        processed_files, failed_files = [], []
        for filename in file_list:
            if not os.path.exists(filename):
                print(f"⚠️ ATENÇÃO: Arquivo '{filename}' não encontrado! Pulando...")
                failed_files.append(filename)
                continue
            print(f"Processando '{filename}'...")
            processed_files.append(filename)
            transcription_data = []
            with open(filename, 'r', encoding='utf-8') as f:
                content = f.read()
                f.seek(0)

                has_timestamps = re.search(r'\[(\d{2}:\d{2}:\d{2}\.\d{3})\s-->\s(\d{2}:\d{2}:\d{2}\.\d{3})\]', content)

                if has_timestamps:
                     for line in f:
                        match = re.match(r'\[(\d{2}:\d{2}:\d{2}\.\d{3})\s-->\s(\d{2}:\d{2}:\d{2}\.\d{3})\]\s*(.*)', line)
                        if match:
                            start_str, end_str, text = match.groups()
                            transcription_data.append({
                                "start": timestamp_to_seconds(start_str),
                                "end": timestamp_to_seconds(end_str),
                                "text": text.strip()
                            })
                else:
                    print(f"   ℹ️ INFO: Arquivo '{filename}' não contém timestamps. O texto será dividido por frases.")
                    sentences = [s.strip() for s in re.split(r'[,.!?\n]+', content) if s.strip()]
                    for text in sentences:
                         transcription_data.append({
                            "start": 0.0,
                            "end": 0.0,
                            "text": text
                        })

            df_individual = pd.DataFrame(transcription_data)
            if not df_individual.empty:
                analises_individuais[filename] = {'df': df_individual, 'full_text': " ".join(df_individual['text'])}

        if analises_individuais:
            print(f"\n✅ Processamento concluído! {len(analises_individuais)} arquivo(s) foram carregados para análise individual.")
        else:
            print("\n⚠️ Nenhum arquivo válido foi processado.")

In [ ]:
# @title 🛠️ 4. Preparação do Texto para Análise
# @markdown > **Execute esta célula antes de prosseguir.**
# @markdown >
# @markdown > Ela realiza o processamento linguístico dos textos, criando as bases de dados necessárias para todas as análises das células seguintes.

import spacy
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Carrega o modelo de linguagem uma única vez
try:
    if 'nlp' not in globals():
        print("Carregando modelo de linguagem (spaCy)...")
        nlp = spacy.load("pt_core_news_lg")
        print("✅ Modelo carregado.")
except Exception as e:
    print(f"❌ Ocorreu um erro ao carregar o modelo spaCy: {e}")

# Verifica se a análise conjunta foi selecionada e prepara os dados
if 'juntar_arquivos_para_analise_conjunta' in globals() and juntar_arquivos_para_analise_conjunta:
    if 'df_transcription' in globals() and not df_transcription.empty:
        print("\n--- PREPARANDO DADOS PARA ANÁLISE CONJUNTA ---")
        # Garante que as variáveis globais sejam criadas ou atualizadas
        full_text = " ".join(df_transcription['text'])
        print("Processando texto combinado (pode demorar em textos longos)...")
        doc = nlp(full_text)
        lemmatized_sents = [" ".join([token.lemma_.lower() for token in nlp(sent)]) for sent in df_transcription['text']]
        df_transcription['lemmatized_text'] = lemmatized_sents
        print("✅ Dados para análise conjunta estão prontos.")
    else:
        print("❌ Nenhum dado para análise conjunta encontrado. Execute a Célula 3A ou 3B primeiro.")

# Verifica se a análise individual foi selecionada e prepara os dados
elif 'analises_individuais' in globals() and analises_individuais:
    print("\n--- PREPARANDO DADOS PARA ANÁLISE INDIVIDUAL ---")
    for filename, data in analises_individuais.items():
        print(f"Processando '{filename}'...")
        # Garante que as variáveis necessárias existam dentro de cada dicionário de arquivo
        data['doc'] = nlp(data['full_text'])
        data['df']['lemmatized_text'] = [" ".join([token.lemma_.lower() for token in nlp(sent)]) for sent in data['df']['text']]
    print("✅ Dados para análise individual estão prontos.")
else:
    print("❌ Nenhum dado de transcrição carregado. Execute a Célula 3A ou 3B primeiro.")

In [ ]:
# @title ☁️ 5. Nuvem de Palavras e Análise de Frequência
# @markdown > A nuvem de palavras é uma representação visual que destaca os termos mais frequentes no texto, permitindo uma rápida identificação dos principais temas.

# @markdown ### 🔹 Parâmetros de Visualização
# @markdown Defina o número máximo de palavras para a nuvem e marque a caixa para ver o gráfico de frequência.
max_words_na_nuvem = 50 #@param {type:"slider", min:20, max:300, step:10}
mostrar_grafico_de_frequencia = True #@param {type:"boolean"}

import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import pandas as pd
from collections import Counter

# --- Funções Auxiliares para Gráficos ---
def plot_word_cloud(text, max_words):
    stopwords = list(spacy.lang.pt.stop_words.STOP_WORDS)
    wordcloud = WordCloud(width=1000, height=500, background_color='white', stopwords=stopwords, collocations=True, max_words=max_words).generate(text)
    plt.figure(figsize=(12, 6))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.show()

# --- FUNÇÃO DO GRÁFICO DE BARRAS ---
def plot_frequency_barchart(text, nlp_model, num_words=25):
    stopwords = list(spacy.lang.pt.stop_words.STOP_WORDS)
    words = [token.text.lower() for token in nlp_model(text) if token.is_alpha and token.text.lower() not in stopwords]
    word_counts = Counter(words).most_common(num_words)

    if not word_counts:
        print("Não há palavras suficientes para gerar o gráfico de frequência.")
        return

    df_freq = pd.DataFrame(word_counts, columns=['Palavra', 'Frequência'])

    plt.figure(figsize=(12, 10))
    ax = sns.barplot(x='Frequência', y='Palavra', data=df_freq, color='skyblue')

    plt.title(f'As {num_words} Palavras Mais Frequentes', size=16)
    plt.xlabel("")
    plt.ylabel("Palavra")

    ax.get_xaxis().set_visible(False)
    sns.despine(left=True, bottom=True)

    for p in ax.patches:
        width = p.get_width()
        ax.text(width + (df_freq['Frequência'].max() * 0.01),
                p.get_y() + p.get_height() / 2,
                f'{int(width)}',
                ha='left',
                va='center',
                color='gray',
                fontsize=11)

    plt.show()

if 'juntar_arquivos_para_analise_conjunta' in globals() and juntar_arquivos_para_analise_conjunta:
    if 'full_text' in globals():
        print("\n--- ANÁLISE CONJUNTA ---")

        print("\n--- ☁️ Nuvem de Palavras ---")
        plot_word_cloud(full_text, max_words_na_nuvem)

        if mostrar_grafico_de_frequencia:
            print("\n--- 📊 Gráfico de Frequência de Palavras ---")
            plot_frequency_barchart(full_text, nlp)

    else:
        print("❌ Nenhum dado de análise conjunta encontrado. Execute a Célula 4 primeiro.")

elif 'analises_individuais' in globals() and analises_individuais:
    print("\n--- ANÁLISE INDIVIDUAL ---")
    for filename, data in analises_individuais.items():
        if 'full_text' in data:
            print(f"\n" + "="*50 + f"\nVisualizações para o arquivo: {filename}\n" + "="*50)

            print(f"\n--- ☁️ Nuvem de Palavras: {filename} ---")
            plot_word_cloud(data['full_text'], max_words_na_nuvem)

            if mostrar_grafico_de_frequencia:
                print(f"\n--- 📊 Gráfico de Frequência: {filename} ---")
                plot_frequency_barchart(data['full_text'], nlp)

else:
    print("❌ Nenhum dado de transcrição carregado. Execute as Células 3B e 4 primeiro.")


In [ ]:
# @title 🏷️ 6. Reconhecimento de Entidades Nomeadas (NER)
# @markdown > O Reconhecimento de Entidades Nomeadas (NER) identifica e classifica nomes de **Pessoas (PER)**, **Locais (LOC)** e **Organizações (ORG)**, sendo útil para mapear rapidamente os principais atores e lugares de um discurso.

# @markdown >
# @markdown > **Importante:** Por ser um processo automatizado, o sistema pode cometer erros, como não identificar uma entidade ou classificá-la incorretamente. Recomenda-se a revisão humana dos resultados para garantir a precisão.
# @markdown >

# @markdown > No modo de análise individual, uma seção ao final aponta quais entidades foram mencionadas em mais de um documento.

from collections import Counter, defaultdict
from IPython.display import display, HTML

# Verifica se a análise conjunta foi selecionada
if 'juntar_arquivos_para_analise_conjunta' in globals() and juntar_arquivos_para_analise_conjunta:
    if 'doc' in globals():
        print("\n--- 🏷️ Reconhecimento de Entidades (Análise Conjunta) ---")
        allowed_labels = {'PER', 'LOC', 'ORG'}
        entities = [(ent.text, ent.label_) for ent in doc.ents if ent.label_ in allowed_labels]
        if not entities:
            print("Nenhuma entidade dos tipos PER, LOC ou ORG foi encontrada.")
        else:
            entity_counts = Counter([ent[0] for ent in entities])
            sorted_entities = sorted(entity_counts.items(), key=lambda item: item[1], reverse=True)

            html_table = "<table><tr><th>Entidade</th><th>Tipo</th><th>Frequência</th></tr>"
            for entity, count in sorted_entities:
                ent_type = [e[1] for e in entities if e[0] == entity][0]
                html_table += f"<tr><td>{entity}</td><td>{ent_type}</td><td>{count}</td></tr>"
            html_table += "</table>"
            display(HTML(html_table))
    else:
        print("❌ Nenhum dado de análise conjunta encontrado. Execute as células anteriores primeiro.")

# Verifica se a análise individual foi selecionada
elif 'analises_individuais' in globals() and analises_individuais:
    print("\n--- ANÁLISE INDIVIDUAL ---")
    all_entity_counts = {}
    entity_to_type = {}

    for filename, data in analises_individuais.items():
        if 'doc' in data:
            print(f"\n--- 🏷️ Reconhecimento de Entidades para: {filename} ---")
            allowed_labels = {'PER', 'LOC', 'ORG'}
            entities = [(ent.text, ent.label_) for ent in data['doc'].ents if ent.label_ in allowed_labels]

            if not entities:
                print("Nenhuma entidade dos tipos PER, LOC ou ORG foi encontrada neste arquivo.")
            else:
                counts = Counter([ent[0] for ent in entities])
                all_entity_counts[filename] = counts
                for entity, label in entities:
                    if entity not in entity_to_type:
                        entity_to_type[entity] = label

                sorted_entities = sorted(counts.items(), key=lambda item: item[1], reverse=True)
                html_table = "<table><tr><th>Entidade</th><th>Tipo</th><th>Frequência</th></tr>"
                for entity, count in sorted_entities:
                    html_table += f"<tr><td>{entity}</td><td>{entity_to_type[entity]}</td><td>{count}</td></tr>"
                html_table += "</table>"
                display(HTML(html_table))

    # --- ANÁLISE COMPARATIVA EM TEXTO ---
    if len(all_entity_counts) > 1:
        print("\n" + "="*50)
        print("--- 🔎 Análise de Entidades em Comum entre Documentos ---")
        print("="*50)

        master_entity_count = Counter()
        for filename in all_entity_counts:
            master_entity_count.update(all_entity_counts[filename].keys())

        common_entities = {entity for entity, count in master_entity_count.items() if count > 1}

        if not common_entities:
            print("\nNenhuma entidade nomeada foi encontrada em mais de um documento.")
        else:
            print("\nAs seguintes entidades foram mencionadas em múltiplos documentos:\n")
            for entity in sorted(list(common_entities)):
                doc_mentions = []
                ent_type = entity_to_type[entity]

                for filename, counts in all_entity_counts.items():
                    if entity in counts:
                        count = counts[entity]
                        plural = "vezes" if count > 1 else "vez"
                        doc_mentions.append(f"no documento '{filename}' ({count} {plural})")

                if len(doc_mentions) == 2:
                    mentions_str = " e ".join(doc_mentions)
                else:
                    mentions_str = ", ".join(doc_mentions[:-1]) + ", e " + doc_mentions[-1]

                print(f"🔹 A entidade '{entity}' (tipo: {ent_type}) foi encontrada {mentions_str}.")
else:
    print("❌ Nenhum dado de transcrição carregado. Execute a Célula 3A ou 3B primeiro.")

In [ ]:
# @title 📈 7. Contagem de Frequência de Palavra Específica
# @markdown > Esta célula conta quantas vezes uma **palavra ou frase** aparece nas transcrições.
# @markdown > Você pode optar por buscar a forma exata ou considerar **variações linguísticas** da palavra ao retornar ela a sua raiz usando lematização.

# @markdown ### 🔹 Defina os parâmetros da sua busca
# @markdown Digite a palavra ou termo que deseja contar nos textos.

# @markdown **Palavra ou Frase para Contar:**
termo_alvo = "rio" # @param {type:"string"}

# @markdown ---
# @markdown **Opções de Análise:**
# @markdown Marque a caixa para contar todas as variações da palavra (ex: buscar por "correr" também contará "correu", "correndo", etc.).
usar_lematizacao = True #@param {type:"boolean"}


from collections import Counter
import re

def contar_frequencia(doc, term, use_lemmas, nlp_model):
    target_term = term.strip().lower()
    if not target_term:
        print("O termo de busca está vazio.")
        return

    if use_lemmas:
        target_form = " ".join([token.lemma_.lower() for token in nlp_model(target_term)])
        target_display = f"{term.strip()} (raiz: {target_form})"
        text_to_search = " ".join([token.lemma_.lower() for token in doc])
    else:
        target_form = target_term
        target_display = target_term
        text_to_search = doc.text.lower()

    count = len(re.findall(r'\b' + re.escape(target_form) + r'\b', text_to_search))

    if count > 0:
        plural = "vezes" if count > 1 else "vez"
        print(f"✅ O termo '{target_display}' foi encontrado {count} {plural}.")
    else:
        print(f"❌ O termo '{target_display}' não foi encontrado.")


if 'juntar_arquivos_para_analise_conjunta' in globals() and juntar_arquivos_para_analise_conjunta:
    if 'doc' in globals():
        print("--- 🔎 Contagem de Frequência para a ANÁLISE CONJUNTA ---")
        contar_frequencia(doc, termo_alvo, usar_lematizacao, nlp)
    else:
        print("❌ Nenhum dado de análise conjunta encontrado. Execute as Células 3 e 4 primeiro.")

elif 'analises_individuais' in globals() and analises_individuais:
    print("--- 🔎 Contagem de Frequência para a ANÁLISE INDIVIDUAL ---")
    for filename, data in analises_individuais.items():
        if 'doc' in data:
            print(f"\n\n--- 📜 Resultados para o arquivo: {filename} ---")
            contar_frequencia(data['doc'], termo_alvo, usar_lematizacao, nlp)
        else:
            print(f"❌ Dados para o arquivo '{filename}' não estão prontos. Execute a Célula 4 primeiro.")
else:
    print("❌ Nenhum dado de transcrição carregado. Execute as Células 3 e 4 primeiro.")


In [ ]:
# @title 🧠 8. Preparação dos Modelos para Análise
# @markdown > **Execute esta célula antes de prosseguir.**
# @markdown >
# @markdown > Ela prepara os modelos de IA necessários tanto para a **Busca Semântica** (na Célula 9) quanto para a **Auto-Codificação Semântica** (na Célula 12).
from sentence_transformers import SentenceTransformer, util
import torch
from collections import Counter

try:
    if 'embedder' not in globals():
        print("Carregando o modelo de busca semântica (SentenceTransformer)...")
        embedder = SentenceTransformer('distiluse-base-multilingual-cased-v1')
        print("✅ Modelo de busca carregado.")

    # --- Análise Conjunta ---
    if 'juntar_arquivos_para_analise_conjunta' in globals() and juntar_arquivos_para_analise_conjunta:
        if 'df_transcription' in globals() and 'doc' in globals():
            print("\nPreparando dados para a ANÁLISE CONJUNTA...")
            # 1. Embeddings de Sentenças (para busca por frase)
            if 'corpus_embeddings' not in globals():
                print(" -> Gerando embeddings de sentenças...")
                corpus_embeddings = embedder.encode(df_transcription['text'].tolist(), convert_to_tensor=True)
            # 2. Embeddings de Palavras (para a nova busca por palavras similares)
            if 'word_embeddings' not in globals():
                print(" -> Gerando embeddings de palavras...")
                words = Counter([token.lemma_.lower() for token in doc if token.is_alpha and not token.is_stop])
                unique_words = list(words.keys())
                vocab_embeddings = embedder.encode(unique_words, convert_to_tensor=True, show_progress_bar=True)
                word_embeddings = {'words': unique_words, 'embeddings': vocab_embeddings}
            print("✅ Dados para análise conjunta estão prontos!")
        else:
            print("❌ Nenhum dado de análise conjunta encontrado. Execute as Células 3 e 4.")

    # --- Análise Individual ---
    elif 'analises_individuais' in globals() and analises_individuais:
        print("\nPreparando dados para a ANÁLISE INDIVIDUAL...")
        for filename, data in analises_individuais.items():
            if 'df' in data and 'doc' in data:
                print(f"\nProcessando '{filename}'...")
                # 1. Embeddings de Sentenças
                if 'corpus_embeddings' not in data:
                    print(" -> Gerando embeddings de sentenças...")
                    data['corpus_embeddings'] = embedder.encode(data['df']['text'].tolist(), convert_to_tensor=True)
                # 2. Embeddings de Palavras
                if 'word_embeddings' not in data:
                    print(" -> Gerando embeddings de palavras...")
                    words = Counter([token.lemma_.lower() for token in data['doc'] if token.is_alpha and not token.is_stop])
                    unique_words = list(words.keys())
                    vocab_embeddings = embedder.encode(unique_words, convert_to_tensor=True, show_progress_bar=False)
                    data['word_embeddings'] = {'words': unique_words, 'embeddings': vocab_embeddings}
        print("\n✅ Dados para análise individual estão prontos!")
    else:
        print("❌ Nenhum dado de transcrição carregado. Execute a Célula 3 e 4 primeiro.")

except Exception as e:
    print(f"❌ Ocorreu um erro ao preparar os modelos de busca: {e}")

In [ ]:
# @title 🔍 9. Busca Interativa (Booleana, Lema e Semântica)
# @markdown > Esta célula permite realizar buscas interativas nos textos carregados.
# @markdown > Você pode buscar por palavras exatas, raízes (lemas), ideias semelhantes ou palavras semanticamente próximas, com foco em temas como meio ambiente e impactos nos rios.

# @markdown > **Tipos de Busca Disponíveis:**
# @markdown >
# @markdown > 🔹 **Busca Exata (Booleana):**
# @markdown > Retorna frases que contenham exatamente os termos digitados.
# @markdown > Permite uso de operadores como **AND**, **OR**, parênteses `( )` e aspas `" "` para frases específicas.
# @markdown > Exemplo: `(rio AND "problema ambiental")`
# @markdown >
# @markdown > 🔹 **Busca por Lema (Booleana):**
# @markdown > Aplica a lógica booleana considerando a raiz das palavras (lema), permitindo capturar variações linguísticas.
# @markdown > Exemplo: `(poluir AND peixe)` também encontra frases com "poluição", "poluindo", "peixes", etc.
# @markdown >
# @markdown > 🔹 **Busca Semântica (por contexto):**
# @markdown > Compara **frases inteiras** por similaridade de significado, útil para encontrar ideias próximas mesmo com vocabulário diferente.
# @markdown > Use **OR** para buscar por múltiplos conceitos.
# @markdown > Exemplo: `desmatamento OR contaminação hídrica`
# @markdown >
# @markdown > 🔹 **Busca Semântica (por palavras similares):**
# @markdown > Encontra frases que contenham **sinônimos ou palavras associadas** ao termo-alvo.
# @markdown > Exemplo: `agrotóxico` pode retornar frases com "veneno", "herbicida", "resíduo tóxico", etc.

# @markdown ---
# @markdown ### 🔹 Defina os parâmetros da sua busca

# @markdown **Termo de Busca:**
# @markdown Digite as palavras, frases ou conceitos que deseja encontrar.
# @markdown → Ex: `(rio AND lixo) OR agrotóxico`
query = "rio"  # @param {type:"string"}

# @markdown **Tipo de Busca:**
# @markdown Escolha entre as estratégias de busca disponíveis.
search_type = "Busca por Lema (Booleana)"  # @param ["Busca Exata (Booleana)", "Busca por Lema (Booleana)", "Busca Semântica (por contexto)", "Busca Semântica (por palavras similares)"]

# @markdown **Nível de Similaridade (para buscas semânticas por contexto):**
# @markdown Define o quão parecida a frase precisa ser com seu termo para ser considerada relevante.
# @markdown → Valores maiores tornam a busca mais precisa (e restrita).
nivel_de_similaridade = 0.2  # @param {type:"slider", min:0.1, max:1.0, step:0.1}

# @markdown **Quantidade de Palavras Similares (para busca por palavras similares):**
# @markdown Número de palavras semanticamente próximas que serão consideradas.
# @markdown → Ex: `rio` pode buscar também por "córrego", "nascente", "águas".
numero_de_palavras_similares = 10  # @param {type:"number"}

# @markdown ---
# @markdown ### 🔹 Parâmetro de Proximidade (Apenas para Buscas com `AND`)
# @markdown Ajusta o tamanho do contexto de busca, definindo a distância máxima entre palavras ligadas por **AND**, mesmo se estiverem em frases diferentes. .
# @markdown → Deixe **0** para exigir que estejam na **mesma frase**.
distancia_maxima_palavras = 100  # @param {type:"number"}


import pandas as pd
from IPython.display import display, HTML
import torch
import re
from sentence_transformers import util

def executar_busca(
    dataframe_alvo, query_str, search_type_str, nlp_model,
    distancia_max=0, embedder_model=None, corpus_embeddings_alvo=None,
    similarity_thr=0.5, numero_similares=10
):
    query = query_str.strip()
    if not query:
        print("ℹ️ Termo de Busca está vazio.")
        return

    def format_timestamp(sec):
        try:
            ms = round((sec - int(sec)) * 1000)
            td = pd.to_timedelta(sec, unit='s')
            h, m, s = td.components.hours + td.components.days*24, td.components.minutes, td.components.seconds
            return f"{h:02d}:{m:02d}:{s:02d}.{ms:03d}"
        except:
            return "00:00:00.000"

    def run_boolean_search(expr, df, lemma=False):
        tokens = re.split(r'(\sAND\s|\sOR\s|\(|\)|"[^"]+")', expr, flags=re.IGNORECASE)
        tokens = [t.strip() for t in tokens if t and t.strip()]
        pd_tokens, keys = [], []
        for t in tokens:
            up = t.upper()
            if up == 'AND':
                pd_tokens.append('&')
            elif up == 'OR':
                pd_tokens.append('|')
            elif t in ('(', ')'):
                pd_tokens.append(t)
            else:
                kw = t.strip('"'); keys.append(kw)
                if lemma:
                    lem = " ".join([w.lemma_ for w in nlp_model(kw.lower())])
                    col, term = 'lemmatized_text', re.escape(lem)
                else:
                    col, term = 'text', re.escape(kw)
                pd_tokens.append(f"df['{col}'].str.contains(r'\\b{term}\\b', case=False, regex=True)")
        mask = eval(" ".join(pd_tokens))
        return df[mask], keys

    html = "<table>"
    found = False
    bool_search = search_type_str.startswith('Busca Exata') or search_type_str.startswith('Busca por Lema')
    lemma_search = (search_type_str == 'Busca por Lema (Booleana)')

    # === BOOLEANA / LEMA ===
    if bool_search:
        clauses = [c.strip() for c in re.split(r'\s+OR\s+', query, flags=re.IGNORECASE)]
        combined_indices = set()
        for clause in clauses:
            if 'AND' in clause.upper() and distancia_max > 0:
                clean = re.sub(r'[\(\)]', '', clause)
                terms = [w.strip().strip('"') for w in re.split(r'\sAND\s', clean, flags=re.IGNORECASE)]
                if lemma_search:
                    search_terms = [" ".join([w.lemma_ for w in nlp_model(t.lower())]) for t in terms]
                    col = 'lemmatized_text'
                else:
                    search_terms = terms; col = 'text'
                for i in range(len(dataframe_alvo)):
                    if i in combined_indices: continue
                    if re.search(r'\b'+re.escape(search_terms[0])+r'\b', dataframe_alvo.iloc[i][col], re.IGNORECASE):
                        window, idxs = [], []
                        for j in range(i, len(dataframe_alvo)):
                            window.append(dataframe_alvo.iloc[j]['text']); idxs.append(j)
                            txt = " ".join(window)
                            if len(txt.split()) > distancia_max + 20: break
                            check = (" ".join([w.lemma_.lower() for w in nlp_model(txt)])
                                     if lemma_search else txt)
                            if all(re.search(r'\b'+re.escape(t)+r'\b', check, re.IGNORECASE)
                                   for t in search_terms):
                                sf = format_timestamp(dataframe_alvo.iloc[idxs[0]]['start'])
                                ef = format_timestamp(dataframe_alvo.iloc[idxs[-1]]['end'])
                                htxt = txt
                                for t in terms:
                                    htxt = re.sub(f'({re.escape(t)})',
                                                  lambda m: f"<mark>{m.group(1)}</mark>",
                                                  htxt, flags=re.IGNORECASE)
                                html += f"<tr><td style='white-space:nowrap;'><b>[{sf} → {ef}]</b></td><td>{htxt}</td></tr>"
                                found = True
                                combined_indices.update(idxs)
                                break
            else:
                df_res, keys = run_boolean_search(clause, dataframe_alvo, lemma_search)
                if not df_res.empty:
                    for _, row in df_res.iterrows():
                        sf = format_timestamp(row['start']); ef = format_timestamp(row['end'])
                        txt = row['text']
                        for k in keys:
                            txt = re.sub(f'({re.escape(k)})',
                                         lambda m: f"<mark>{m.group(1)}</mark>",
                                         txt, flags=re.IGNORECASE)
                        html += f"<tr><td style='white-space:nowrap;'><b>[{sf} → {ef}]</b></td><td>{txt}</td></tr>"
                        found = True

    # === SEMÂNTICA (por contexto) ===
    elif search_type_str == 'Busca Semântica (por contexto)':
        print(f"Semântica: '{query}' (thr={similarity_thr})")
        clauses = [c.strip() for c in re.split(r'\s+OR\s+', query, flags=re.IGNORECASE)]
        scores = {}
        for c in clauses:
            df_ex = dataframe_alvo[dataframe_alvo['text'].str.contains(re.escape(c), case=False, regex=True)]
            for idx, row in df_ex.iterrows():
                scores[idx] = (1.0, c)
            emb = embedder_model.encode(c, convert_to_tensor=True)
            sims = util.cos_sim(emb, corpus_embeddings_alvo)[0]
            for tidx in torch.where(sims >= similarity_thr)[0].tolist():
                i, sc = int(tidx), float(sims[tidx])
                if i not in scores or sc > scores[i][0]:
                    scores[i] = (sc, c)
        for i, (sc, cq) in sorted(scores.items(), key=lambda x: -x[1][0]):
            row = dataframe_alvo.iloc[i]
            sf = format_timestamp(row['start']); ef = format_timestamp(row['end'])
            if sc == 1.0:
                txt = re.sub(f'({re.escape(cq)})',
                             lambda m: f"<mark>{m.group(1)}</mark>",
                             row['text'], flags=re.IGNORECASE)
                tag = ""
            else:
                txt = row['text']; tag = f" (sim={sc:.2f})"
            html += f"<tr><td style='white-space:nowrap;'><b>[{sf} → {ef}]</b>{tag}</td><td>{txt}</td></tr>"
            found = True

    # === SEMÂNTICA (por palavras similares) ===
    elif search_type_str == 'Busca Semântica (por palavras similares)':
        if 'word_embeddings' not in globals():
            print("❌ Não achei 'word_embeddings' em globals(). Execute a célula de preparação de embeddings de palavras.")
        else:
            we = globals()['word_embeddings']
            emb = embedder_model.encode(query, convert_to_tensor=True)
            sims = util.cos_sim(emb, we['embeddings'])[0]
            topk = torch.topk(sims, k=min(numero_similares, len(we['words'])))
            similar_words = [we['words'][i] for _, i in zip(topk[0], topk[1])]
            print(f"Palavras similares a '{query}': {', '.join(similar_words)}")
            pattern = r'\b(' + '|'.join(re.escape(w) for w in similar_words) + r')\b'
            df_res = dataframe_alvo[dataframe_alvo['text'].str.contains(pattern, case=False, regex=True)]
            if df_res.empty:
                print("Nenhum resultado encontrado.")
            else:
                for _, row in df_res.iterrows():
                    sf = format_timestamp(row['start']); ef = format_timestamp(row['end'])
                    txt = re.sub(pattern, lambda m: f"<mark>{m.group(1)}</mark>", row['text'], flags=re.IGNORECASE)
                    html += f"<tr><td style='white-space:nowrap;'><b>[{sf} → {ef}]</b></td><td>{txt}</td></tr>"
                found = True

    if not found:
        print("Nenhum resultado encontrado.")
    html += "</table>"
    display(HTML(html))

# --- Execução da Busca ---
if 'juntar_arquivos_para_analise_conjunta' in globals() and juntar_arquivos_para_analise_conjunta:
    if 'df_transcription' in globals() and 'corpus_embeddings' in globals():
        print("--- Análise Conjunta ---")
        executar_busca(
            df_transcription, query, search_type, nlp,
            distancia_maxima_palavras, embedder, corpus_embeddings, similarity_threshold,
            numero_de_palavras_similares
        )
    else:
        print("❌ Execute Células 3A ou 3B, 4 e 8 antes.")
elif 'analises_individuais' in globals():
    print("--- Análise Individual ---")
    for fn, data in analises_individuais.items():
        print(f"\n--- {fn} ---")
        if 'df' in data and 'corpus_embeddings' in data:
            if 'word_embeddings' in data:
                globals()['word_embeddings'] = data['word_embeddings']
            executar_busca(
                data['df'], query, search_type, nlp,
                distancia_maxima_palavras, embedder,
                data['corpus_embeddings'], similarity_threshold,
                numero_de_palavras_similares
            )
        else:
            print(f"❌ Execute células 4 e 8 para '{fn}'.")
else:
    print("❌ Defina primeiro o modo de análise (Célula 3B).")

In [ ]:
# @title 🕸️ 10. Grafo de Coocorrência e Tabela de Frequências
# @markdown > O Grafo de Coocorrência é um "mapa de ideias" que mostra quais palavras importantes aparecem frequentemente juntas no mesmo contexto. Sua utilidade é revelar as principais associações de termos e os grupos temáticos do discurso.

# @markdown ### 🔹 Parâmetros do Grafo
# @markdown Defina os parâmetros para a geração do grafo.
top_n_palavras = 25 #@param {type:"slider", min:10, max:50, step:5}
usar_lemas_no_grafo = True #@param {type:"boolean"}
# @markdown ---
# @markdown ### 🔹 Definição de Contexto
# @markdown Defina o tamanho (em palavras) do contexto a ser analisado para encontrar coocorrências. **Deixe em 0 para usar as frases originais.**
tamanho_do_contexto = 0 #@param {type:"slider", min:0, max:100, step:5}
# @markdown ---
# @markdown ### 🔹 Filtro de Relevância
# @markdown Defina o número mínimo de vezes que duas palavras precisam aparecer juntas para criar uma conexão.
forca_minima_da_conexao = 2 #@param {type:"slider", min:1, max:20, step:1}
# @markdown ---
# @markdown ### 🔹 Opções de Exibição
# @markdown Marque a caixa para exibir também a tabela com a frequência de cada par.
mostrar_tabela_de_frequencia = False #@param {type:"boolean"}


import networkx as nx
from itertools import combinations
from collections import Counter
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display, HTML

# --- Função do Grafo ---
def plot_cooccurrence_graph_and_table(doc, top_n=25, use_lemmas=True, chunk_size=0, min_weight=2, show_table=True):

    if use_lemmas:
        print(" -> Analisando coocorrências entre LEMAS (raízes das palavras).")
        all_tokens = [token.lemma_.lower() for token in doc if token.pos_ in ('NOUN', 'PROPN', 'ADJ', 'ADV') and not token.is_stop and not token.is_punct]
    else:
        print(" -> Analisando coocorrências entre PALAVRAS originais.")
        all_tokens = [token.text.lower() for token in doc if token.pos_ in ('NOUN', 'PROPN', 'ADJ', 'ADV') and not token.is_stop and not token.is_punct]

    most_common_words = [word for word, count in Counter(all_tokens).most_common(top_n)]

    co_occurrences = []

    if chunk_size == 0:
        print(" -> Contexto: Dentro da mesma frase (delimitada pelo Whisper/spaCy).")
        if use_lemmas:
            for sentence in doc.sents:
                sent_tokens = [token.lemma_.lower() for token in sentence if token.lemma_.lower() in most_common_words]
                co_occurrences.extend(list(combinations(sorted(list(set(sent_tokens))), 2)))
        else:
            for sentence in doc.sents:
                sent_tokens = [token.text.lower() for token in sentence if token.text.lower() in most_common_words]
                co_occurrences.extend(list(combinations(sorted(list(set(sent_tokens))), 2)))
    else:
        print(f" -> Contexto: Trechos de {chunk_size} palavras.")
        filtered_tokens = [token for token in all_tokens if token in most_common_words]

        for i in range(0, len(filtered_tokens), chunk_size):
            chunk = filtered_tokens[i:i + chunk_size]
            co_occurrences.extend(list(combinations(sorted(list(set(chunk))), 2)))

    if not co_occurrences:
        print("\nNão foi possível gerar o grafo e a tabela: nenhuma coocorrência encontrada.")
        return

    co_occurrence_counts = Counter(co_occurrences)

    # --- Geração do Grafo ---
    G = nx.Graph()
    for (word1, word2), weight in co_occurrence_counts.items():
        if weight >= min_weight:
            G.add_edge(word1, word2, weight=weight)

    if G.number_of_nodes() == 0:
        print(f"\nNão foi possível gerar o grafo: nenhuma coocorrência encontrada com força mínima de {min_weight}.")
    else:
        plt.figure(figsize=(16, 16))
        pos = nx.spring_layout(G, k=0.9, iterations=75, seed=42)
        edge_widths = [d['weight'] / 2 for (u, v, d) in G.edges(data=True)]

        degrees = [G.degree(n) for n in G.nodes()]
        if degrees:
            min_degree, max_degree = min(degrees), max(degrees)
            min_size, max_size = 500, 5000

            if max_degree == min_degree:
                node_sizes = [1500] * len(G.nodes())
            else:
                node_sizes = [
                    min_size + (degree - min_degree) * (max_size - min_size) / (max_degree - min_degree)
                    for degree in degrees
                ]
        else:
            node_sizes = []

        nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color='skyblue', alpha=0.9)
        nx.draw_networkx_edges(G, pos, width=edge_widths, alpha=0.5, edge_color='gray')
        nx.draw_networkx_labels(G, pos, font_size=11, font_family='sans-serif')

        plt.title(f'Grafo de Coocorrência dos {top_n} Termos Mais Comuns', size=18)
        plt.box(False)
        plt.show()

    # --- Geração da Tabela ---
    if show_table:
        print("\n---  Tabela de Frequência de Coocorrências ---")
        sorted_counts = co_occurrence_counts.most_common()
        if not sorted_counts:
            print("Nenhuma coocorrência para exibir na tabela.")
            return

        df_data = [(item[0][0], item[0][1], item[1]) for item in sorted_counts]
        df_cooc = pd.DataFrame(df_data, columns=['Palavra 1', 'Palavra 2', 'Frequência'])

        pd.set_option('display.max_rows', None)
        display(df_cooc)
        pd.reset_option('display.max_rows')

# --- Execução Principal ---
if 'juntar_arquivos_para_analise_conjunta' in globals() and juntar_arquivos_para_analise_conjunta:
    if 'doc' in globals():
        print("\n--- 🕸️ Grafo e Tabela de Coocorrência (Análise Conjunta) ---")
        plot_cooccurrence_graph_and_table(doc, top_n=top_n_palavras, use_lemmas=usar_lemas_no_grafo, chunk_size=tamanho_do_contexto, min_weight=forca_minima_da_conexao, show_table=mostrar_tabela_de_frequencia)
    else:
        print("❌ Nenhum dado de análise conjunta encontrado. Execute a Célula 4 primeiro.")

elif 'analises_individuais' in globals() and analises_individuais:
    print("\n--- 🕸️ Grafo e Tabela de Coocorrência (Análise Individual) ---")
    for filename, data in analises_individuais.items():
        if 'doc' in data:
            print(f"\n--- Resultados para: {filename} ---")
            plot_cooccurrence_graph_and_table(data['doc'], top_n=top_n_palavras, use_lemmas=usar_lemas_no_grafo, chunk_size=tamanho_do_contexto, min_weight=forca_minima_da_conexao, show_table=mostrar_tabela_de_frequencia)
else:
    print("❌ Nenhum dado de transcrição carregado. Execute as Células 3B e 4 primeiro.")



In [ ]:
# @title 🔎 11. Análise de Colocação (Palavras Vizinhas)
# @markdown > Esta análise revela quais palavras (vizinhas) aparecem com mais frequência antes ou depois de um termo específico.

# @markdown > Sua utilidade é entender o contexto de uso de uma palavra e descobrir suas associações e frases mais comuns no discurso.

# @markdown ### 🔹 Defina os parâmetros da sua busca
# @markdown Digite a palavra que deseja analisar e escolha as opções de busca.

# @markdown **Palavra-Alvo:**
palavra_alvo = "rio" # @param {type:"string"}

# @markdown ---
# @markdown **Opções de Análise:**
# @markdown Marque a caixa para agrupar as palavras vizinhas por sua raiz (lema). Ex: "longa" e "longo" serão contadas juntas como "longo".
agrupar_palavras_vizinhas_pela_raiz = True #@param {type:"boolean"}

# @markdown ---
# @markdown **Direção da Busca:**
# @markdown Escolha se quer ver as palavras que vêm antes, depois ou em ambas as direções do seu termo.
direcao_busca = "Ambas as direções" # @param ["Direita (palavras seguintes)", "Esquerda (palavras anteriores)", "Ambas as direções"]

# @markdown ---
# @markdown **Filtros para Palavras Vizinhas:**
# @markdown Selecione as classes gramaticais que você deseja ver na análise.
considerar_substantivos = True #@param {type:"boolean"}
considerar_adjetivos = True #@param {type:"boolean"}
considerar_adverbios = True #@param {type:"boolean"}
considerar_verbos = True #@param {type:"boolean"}

# @markdown ---
# @markdown **Opções de Exibição:**
mostrar_timestamps = False #@param {type:"boolean"}


import pandas as pd
from IPython.display import display, HTML
from collections import Counter, defaultdict

def format_timestamp_display(seconds):
    try:
        milliseconds = round((seconds - int(seconds)) * 1000)
        td = pd.to_timedelta(seconds, unit='s')
        h, m, s = td.components.hours + (td.components.days * 24), td.components.minutes, td.components.seconds
        return f"[{h:02d}:{m:02d}:{s:02d}.{milliseconds:03d}]"
    except:
        return "[00:00:00.000]"

def analisar_colocacoes(dataframe_alvo, target_word, group_by_lemma, nlp_model, pos_filters, direction, show_ts):
    target_word_processed = target_word.strip().lower()
    if not target_word_processed:
        print("Palavra-alvo está vazia.")
        return

    allowed_pos = set()
    if pos_filters.get('substantivos'): allowed_pos.update(['NOUN', 'PROPN'])
    if pos_filters.get('adjetivos'): allowed_pos.add('ADJ')
    if pos_filters.get('adverbios'): allowed_pos.add('ADV')
    if pos_filters.get('verbos'): allowed_pos.add('VERB')

    if not allowed_pos:
        print("⚠️ Nenhuma classe gramatical foi selecionada para a análise.")
        return

    collocations = defaultdict(list)

    for index, row in dataframe_alvo.iterrows():
        row_doc = nlp_model(row['text'])
        for i, token in enumerate(row_doc):
            if token.text.lower() == target_word_processed:

                # --- LÓGICA PARA OLHAR PARA A DIREITA ---
                if direction in ["Direita (palavras seguintes)", "Ambas as direções"]:
                    for next_token_index in range(i + 1, len(row_doc)):
                        next_token = row_doc[next_token_index]
                        if next_token.pos_ in allowed_pos and not next_token.is_stop and not next_token.is_punct:
                            key = next_token.lemma_.lower() if group_by_lemma else next_token.text.lower()
                            collocations[key].append(row['start'])
                            break
                        elif next_token.is_punct and next_token.text in ['.', '!', '?']:
                            break

                # --- LÓGICA PARA OLHAR PARA A ESQUERDA ---
                if direction in ["Esquerda (palavras anteriores)", "Ambas as direções"]:
                    for prev_token_index in range(i - 1, -1, -1):
                        prev_token = row_doc[prev_token_index]
                        if prev_token.pos_ in allowed_pos and not prev_token.is_stop and not prev_token.is_punct:
                            key = prev_token.lemma_.lower() if group_by_lemma else prev_token.text.lower()
                            collocations[key].append(row['start'])
                            break
                        elif prev_token.is_punct and prev_token.text in ['.', '!', '?']:
                            break

    if not collocations:
        print("Nenhuma colocação encontrada para o termo e filtros especificados.")
        return

    sorted_collocations = sorted(collocations.items(), key=lambda item: len(item[1]), reverse=True)

    col_name = "Raiz da Palavra (Lema)" if group_by_lemma else "Palavra Vizinha"
    df_data = []

    for word, timestamps in sorted_collocations:
        freq = len(timestamps)
        row_data = {col_name: word, 'Frequência': freq}
        if show_ts:
            timestamps_str = ", ".join([format_timestamp_display(ts) for ts in sorted(list(set(timestamps)))])
            row_data['Timestamps'] = timestamps_str
        df_data.append(row_data)

    columns = [col_name, 'Frequência', 'Timestamps'] if show_ts else [col_name, 'Frequência']
    df_colloc = pd.DataFrame(df_data, columns=columns)

    print(f"Encontradas {len(df_colloc)} palavras vizinhas únicas.")
    pd.set_option('display.max_rows', None); pd.set_option('display.max_colwidth', None)
    display(df_colloc)
    pd.reset_option('display.max_rows'); pd.reset_option('display.max_colwidth')

# --- Bloco de Execução Principal ---
pos_filters_selection = {
    "substantivos": considerar_substantivos,
    "adjetivos": considerar_adjetivos,
    "adverbios": considerar_adverbios,
    "verbos": considerar_verbos
}

if 'juntar_arquivos_para_analise_conjunta' in globals() and juntar_arquivos_para_analise_conjunta:
    if 'df_transcription' in globals():
        print("--- 🔎 Análise de Colocação para a ANÁLISE CONJUNTA ---")
        analisar_colocacoes(df_transcription, palavra_alvo, agrupar_palavras_vizinhas_pela_raiz, nlp, pos_filters_selection, direcao_busca, mostrar_timestamps)
    else:
        print("❌ Nenhum dado de análise conjunta encontrado. Execute as Células 3 e 4 primeiro.")

elif 'analises_individuais' in globals() and analises_individuais:
    print("--- 🔎 Análise de Colocação para a ANÁLISE INDIVIDUAL ---")
    for filename, data in analises_individuais.items():
        if 'df' in data:
            print(f"\n\n--- 📜 Resultados para o arquivo: {filename} ---")
            analisar_colocacoes(data['df'], palavra_alvo, agrupar_palavras_vizinhas_pela_raiz, nlp, pos_filters_selection, direcao_busca, mostrar_timestamps)
        else:
            print(f"❌ Dados para o arquivo '{filename}' não estão prontos. Execute a Célula 4 primeiro.")
else:
    print("❌ Nenhum dado de transcrição carregado. Execute as Células 3 e 4 primeiro.")

In [ ]:
# @title 📚 12.1. Definição do Livro de Códigos e Auto-Codificação
# @markdown > Defina seus códigos e os conceitos para a auto-codificação semântica.

# @markdown ### 🔹 1. Códigos para Codificação Manual
# @markdown Liste todos os seus códigos/temas, separados por **ponto (.)**.
lista_de_codigos_manuais = "Exemplo 1. Exemplo 2. Exemplo 3. Exemplo 4" # @param {type:"string"}

# @markdown ---
# @markdown ### 🔹 2. Dicionário para Auto-Codificação Semântica (Opcional)
# @markdown > **Formato:** `CÓDIGO_1: conceito A, conceito B . CÓDIGO_2: conceito C`
# @markdown > (Use **ponto (.)** para separar diferentes códigos e **vírgula (,)** para múltiplos conceitos dentro de um código).

# @markdown >Obs: Requer execução da célula 8.
dicionario_auto_codificacao = "" # @param {type:"string"}

# @markdown ---
# @markdown ### 🔹 3. Parâmetros da Auto-Codificação
# @markdown Defina o quão similar uma frase precisa ser do conceito para ser auto-codificada.
# @markdown > **Recomendação:** Comece com um valor em torno de 0.6. Valores mais altos são mais precisos, mas podem encontrar menos resultados.
similaridade_auto_codificacao = 0.45 #@param {type:"slider", min:0.1, max:1.0, step:0.05}


import pandas as pd
import re
from sentence_transformers import util
import torch

print("Definindo o livro de códigos...")
manual_codes = [code.strip() for code in re.split(r'\s*\.\s*', lista_de_codigos_manuais) if code.strip()]

auto_code_dict = {}
for entry in re.split(r'\s*\.\s*', dicionario_auto_codificacao.strip()):
    if ':' in entry:
        code, concepts_str = entry.split(':', 1)
        concepts = [concept.strip() for concept in re.split(r'\s*,\s*', concepts_str) if concept.strip()]
        if code.strip() and concepts:
            auto_code_dict[code.strip()] = concepts

all_codes = sorted(list(set(manual_codes + list(auto_code_dict.keys()))))
print(f"✅ Códigos definidos: {all_codes}")


def auto_codificar_semantica(df, dictionary, embedder_model, sent_embeddings, threshold):
    print("\nIniciando auto-codificação semântica...")
    df['codigos'] = [[] for _ in range(len(df))]

    all_concepts = [concept for concepts in dictionary.values() for concept in concepts]
    if not all_concepts:
        print("Dicionário de auto-codificação está vazio. Pulando esta etapa.")
        return df

    concept_embeddings = embedder_model.encode(all_concepts, convert_to_tensor=True)
    similarity_matrix = util.cos_sim(sent_embeddings, concept_embeddings)

    concept_idx = 0
    for code, concepts in dictionary.items():
        for concept in concepts:
            similar_sentences_indices = torch.where(similarity_matrix[:, concept_idx] >= threshold)[0]

            for i_tensor in similar_sentences_indices:
                i = i_tensor.item()
                if code not in df.at[i, 'codigos']:
                    df.at[i, 'codigos'].append(code)
            concept_idx += 1

    print("✅ Auto-codificação concluída.")
    return df

def preparar_dataframe_para_codificacao(df_original, corpus_embeds):
    df = df_original.copy()

    if auto_code_dict:
        if 'embedder' in globals() and corpus_embeds is not None:
             df = auto_codificar_semantica(df, auto_code_dict, embedder, corpus_embeds, similaridade_auto_codificacao)
        else:
            print("\n⚠️ AVISO: Auto-codificação não foi executada. É preciso executar a Célula 8 (Preparação dos Modelos de Busca) primeiro.")
    elif 'codigos' not in df.columns or not all(isinstance(x, list) for x in df['codigos']):
        df['codigos'] = [[] for _ in range(len(df))]

    return df

if 'juntar_arquivos_para_analise_conjunta' in globals() and juntar_arquivos_para_analise_conjunta:
    if 'df_transcription' in globals() and 'corpus_embeddings' in globals():
        df_transcription = preparar_dataframe_para_codificacao(df_transcription, corpus_embeddings)
        print("\nPronto para a codificação da ANÁLISE CONJUNTA.")
elif 'analises_individuais' in globals() and analises_individuais:
    for filename, data in analises_individuais.items():
        if 'df' in data and 'corpus_embeddings' in data:
            print(f"\n--- Preparando '{filename}' ---")
            data['df'] = preparar_dataframe_para_codificacao(data['df'], data['corpus_embeddings'])
    print("\nPronto para a codificação da ANÁLISE INDIVIDUAL.")
else:
    print("❌ Nenhum dado carregado. Execute as Células 3, 4 e 8 primeiro.")

In [ ]:
# @title 💻 12.2. Interface de Codificação Manual
import ipywidgets as widgets
from IPython.display import display, clear_output, Javascript
import pandas as pd

def format_timestamp(seconds):
    try:
        milliseconds = round((seconds - int(seconds)) * 1000)
        td = pd.to_timedelta(seconds, unit='s')
        h, m, s = td.components.hours + (td.components.days * 24), td.components.minutes, td.components.seconds
        return f"{h:02d}:{m:02d}:{s:02d}.{milliseconds:03d}"
    except: return "00:00:00.000"

target_df = None
is_plain_text_mode = False

if 'juntar_arquivos_para_analise_conjunta' in globals() and juntar_arquivos_para_analise_conjunta:
    if 'df_transcription' in globals():
        print("Iniciando a codificação para a ANÁLISE CONJUNTA.")
        target_df = df_transcription
        if len(target_df) > 1 and target_df['start'].iloc[1] == 1.0: is_plain_text_mode = True

elif 'analises_individuais' in globals() and analises_individuais:
    file_selector = widgets.Dropdown(options=list(analises_individuais.keys()), description='Arquivo:', disabled=False)
    print("Iniciando a codificação para a ANÁLISE INDIVIDUAL.")
    print("Selecione o arquivo que deseja codificar:")
    display(file_selector)
    target_df = analises_individuais[file_selector.value]['df']
    if len(target_df) > 1 and target_df['start'].iloc[1] == 1.0: is_plain_text_mode = True
else:
    print("❌ Nenhum dado carregado para codificar.")

if target_df is not None and 'all_codes' in locals() and all_codes:

    if 'coding_indices' not in globals():
        coding_indices = {filename: 0 for filename in analises_individuais.keys()}
        coding_indices['conjunta'] = 0

    def get_current_state():
        if 'juntar_arquivos_para_analise_conjunta' in globals() and juntar_arquivos_para_analise_conjunta:
            key = 'conjunta'; df = df_transcription
        else:
            key = file_selector.value; df = analises_individuais[key]['df']
        return key, df, coding_indices.get(key, 0)

    current_key, current_df, current_index = get_current_state()
    total_segments = len(current_df)

    segment_text_area = widgets.HTML(layout=widgets.Layout(height='auto', border='1px solid lightgray', padding='10px', margin='10px 0'))
    checkboxes = {code: widgets.Checkbox(description=code, value=False, indent=False) for code in all_codes}
    checkbox_container = widgets.VBox(children=list(checkboxes.values()))
    full_text_display = widgets.HTML(layout=widgets.Layout(height='200px', border='1px solid lightgray', padding='10px', overflow='scroll'))
    progress_bar = widgets.IntProgress(value=0, min=0, max=total_segments, description='Progresso:', bar_style='info')
    prev_button = widgets.Button(description="Anterior", icon='arrow-left')
    next_button = widgets.Button(description="Próximo / Salvar", icon='arrow-right', button_style='success')
    new_code_text = widgets.Text(placeholder='Digite um novo código...')
    add_code_button = widgets.Button(description="Adicionar Código")
    clear_button = widgets.Button(description="Limpar Codificações Deste Arquivo", icon='trash', button_style='danger')
    output_area = widgets.Output()

    def update_segment_view(index, df_to_use):
        if not (0 <= index < len(df_to_use)): return

        progress_bar.value = index + 1
        progress_bar.max = len(df_to_use)
        segment_row = df_to_use.iloc[index]
        saved_codes = segment_row.get('codigos', [])

        is_plain = len(df_to_use) > 1 and df_to_use['start'].iloc[1] == 1.0
        header = f"<b>Segmento {index + 1} de {len(df_to_use)}</b>" if is_plain else f"<b>[{format_timestamp(segment_row['start'])} --> {format_timestamp(segment_row['end'])}]</b>"
        segment_text_area.value = f"{header}<br>{segment_row['text']}"

        html_lines = []
        for i, text in enumerate(df_to_use['text']):
            line_id = f"segment-scroll-{i}"
            if i == index:
                html_lines.append(f"<span id='{line_id}'><mark>{text}</mark></span>")
            else:
                html_lines.append(f"<span id='{line_id}'>{text}</span>")

        scroll_script = f"""
        <script>
            setTimeout(function() {{
                var element = document.getElementById('segment-scroll-{index}');
                if (element) {{
                    element.scrollIntoView({{ behavior: 'smooth', block: 'center', inline: 'nearest' }});
                }}
            }}, 150);
        </script>
        """
        full_text_display.value = "<br>".join(html_lines)

        for code, cb in checkboxes.items():
            cb.value = code in saved_codes

    def save_codes_and_go_to(offset):
        key, df, current_idx = get_current_state()
        selected_codes = [cb.description for cb in checkboxes.values() if cb.value]
        df.at[current_idx, 'codigos'] = selected_codes
        new_index = current_idx + offset
        if 0 <= new_index < len(df):
            coding_indices[key] = new_index
            update_segment_view(new_index, df)
        elif new_index >= len(df):
            with output_area: clear_output(); print("✅ Codificação de todos os segmentos concluída para este arquivo!")

    def on_add_code_button_clicked(b):
        new_code = new_code_text.value.strip()
        if new_code and new_code not in checkboxes:
            checkboxes[new_code] = widgets.Checkbox(description=new_code, value=True, indent=False)
            checkbox_container.children = list(checkboxes.values())
            new_code_text.value = ""

    def on_clear_button_clicked(b):
        with output_area:
            clear_output()
            key, df, current_idx = get_current_state()
            print(f"Limpando todas as {len(df['codigos'])} codificações do arquivo '{key}'...")
            df['codigos'] = [[] for _ in range(len(df))]
            update_segment_view(current_idx, df)
            print("✅ Todas as codificações foram removidas.")

    def on_file_change(change):
        key, df, index = get_current_state()
        update_segment_view(coding_indices.get(change.new, 0), analises_individuais[change.new]['df'])

    next_button.on_click(lambda b: save_codes_and_go_to(1))
    prev_button.on_click(lambda b: save_codes_and_go_to(-1))
    add_code_button.on_click(on_add_code_button_clicked)
    clear_button.on_click(on_clear_button_clicked)

    if 'file_selector' in locals():
      file_selector.observe(on_file_change, names='value')

    update_segment_view(current_index, current_df)

    ui = widgets.VBox([
        progress_bar, segment_text_area,
        widgets.Label("Selecione os códigos aplicáveis:"), checkbox_container,
        widgets.HBox([widgets.Label("Adicionar novo código:"), new_code_text, add_code_button]),
        widgets.HBox([prev_button, next_button, clear_button]),
        output_area,
        widgets.Label("Contexto do Texto Completo (segmento atual em destaque):"), full_text_display
    ])
    display(ui)

In [ ]:
# @title 📊 12.3. Análise dos Resultados da Codificação
# @markdown > Esta célula analisa os códigos que você aplicou, mostrando a frequência de cada tema e permitindo que você recupere todas as frases associadas a um código específico.

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import ipywidgets as widgets
from IPython.display import display, HTML

def analisar_codigos(df, title):
    print("\n" + "="*50 + f"\nAnalisando Resultados para: {title}\n" + "="*50)

    if 'codigos' not in df.columns or not all(isinstance(x, list) for x in df['codigos']):
        print("Coluna 'codigos' não encontrada ou em formato inválido. Execute a Célula 12.1 e 12.2 primeiro.")
        return

    all_codes_applied = [code for sublist in df['codigos'] for code in sublist]
    if not all_codes_applied:
        print("Nenhum código foi aplicado a este documento ainda.")
        return

    code_counts = Counter(all_codes_applied)
    df_freq = pd.DataFrame(code_counts.most_common(), columns=['Código', 'Frequência'])

    print("\n--- 📊 Frequência dos Códigos ---")
    plt.figure(figsize=(12, 8))
    ax = sns.barplot(x='Frequência', y='Código', data=df_freq, color='skyblue')
    ax.get_xaxis().set_visible(False)
    sns.despine(left=True, bottom=True)
    plt.title('Frequência de Cada Código Aplicado', size=16)
    for p in ax.patches:
        width = p.get_width()
        ax.text(width + (df_freq['Frequência'].max() * 0.01), p.get_y() + p.get_height() / 2, f'{int(width)}', ha='left', va='center', color='gray')
    plt.show()

    print("\n--- 📝 Recuperar Segmentos por Código ---")
    print("Selecione um código para ver todas as frases associadas a ele.")

    code_selector = widgets.Dropdown(options=sorted(list(code_counts.keys())))
    button = widgets.Button(description="Mostrar Frases")
    output = widgets.Output()

    def on_button_click(b):
        with output:
            clear_output()
            selected_code = code_selector.value
            filtered_df = df[df['codigos'].apply(lambda codes: selected_code in codes)].copy()

            merged_data = []
            if not filtered_df.empty:
                current_group = []
                for index, row in filtered_df.iterrows():
                    if not current_group or index == current_group[-1]['index'] + 1:
                        current_group.append({'index': index, 'row': row})
                    else:
                        merged_data.append(current_group)
                        current_group = [{'index': index, 'row': row}]
                if current_group:
                    merged_data.append(current_group)
            # --------------------------------------------------

            if not merged_data:
                print(f"Nenhuma frase encontrada para o código: '{selected_code}'")
            else:
                html_table = "<table>"
                is_plain = len(df) > 1 and df['start'].iloc[1] == 1.0
                for group in merged_data:
                    start_row, end_row = group[0]['row'], group[-1]['row']
                    if is_plain:
                        header = f"<b>Segmento {group[0]['index'] + 1} a {group[-1]['index'] + 1}</b>"
                    else:
                        header = f"<b>[{format_timestamp(start_row['start'])} --> {format_timestamp(end_row['end'])}]</b>"

                    combined_text = " ".join([item['row']['text'] for item in group])
                    html_table += f"<tr><td style='white-space:nowrap; vertical-align: top;'>{header}</td><td>{combined_text}</td></tr>"
                html_table += "</table>"
                display(HTML(html_table))

    button.on_click(on_button_click)
    display(widgets.HBox([code_selector, button]), output)

if 'juntar_arquivos_para_analise_conjunta' in globals() and juntar_arquivos_para_analise_conjunta:
    if 'df_transcription' in globals():
        analisar_codigos(df_transcription, "Análise Conjunta")
elif 'analises_individuais' in globals() and analises_individuais:
    for filename, data in analises_individuais.items():
        if 'df' in data:
            analisar_codigos(data['df'], filename)
else:
    print("❌ Nenhum dado de transcrição carregado.")